# 1. Setup

## 1.1 Install Dependencies

In [ ]:
!pip install tensorflow opencv-python matplotlib

## 1.2 Import Dependencies

In [ ]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

## 1.3 Set GPU

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
len(gpus)

## 1.4 Create Folder Structure

In [ ]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [ ]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

# 2. Collect Positive and Anchors


## 2.1 Untar Labelled Faces in the wild dataset

In [ ]:
# !tar -xf lfw.tgz

In [ ]:
# for directory in os.listdir('lfw'):
#   dir_path = os.path.join('lfw', directory)
#   for file in os.listdir(dir_path):
#     EX_PATH = os.path.join(dir_path, file)
#     NEW_PATH = os.path.join(NEG_PATH, file)
#     os.replace(EX_PATH, NEW_PATH)


## 2.2 Collect Positive and Anchor

In [ ]:
import uuid

In [ ]:
'{}.jpg'.format(uuid.uuid1())

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    frame = frame[120:120+250,200:200+250,:]
    
    # Collect anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        #Create the unique file path
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        #Write out anchor image
        cv2.imwrite(imgname, frame)
    
    #Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        #Create the unique file path
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        #Write out anchor image
        cv2.imwrite(imgname, frame)
    
    cv2.imshow('Image Collection', frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
frame[:250,:250,:].shape

In [ ]:
plt.imshow(frame)

# 3. Load and Preprocess Images

## 3.1 Get Image Directories

In [ ]:
anchor = tf.data.Dataset.list_files(ANC_PATH + '\*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH + '\*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH + '\*.jpg').take(300)

In [ ]:
dir_test = anchor.as_numpy_iterator()

In [ ]:
dir_test.next()

## 3.2 Preprocessing - Scaling and Resize

In [ ]:
def preprocess(file_path):
    
    #Read in image from file path
    byte_image = tf.io.read_file(file_path)
    #Load in the image
    img = tf.io.decode_jpeg(byte_image)
    
    #Preprocessing steps - resizing the image to be 100 x 100 x 3
    img = tf.image.resize(img, (100, 100))
    #Scale the image to be between 0 and 1
    img = img / 255.0
    
    #Returning the image
    return img

In [ ]:
img = preprocess('data\\anchor\\13000282-ea7b-11ee-99ce-c89402f529ca.jpg')

In [ ]:
img.numpy().max()

In [ ]:
plt.imshow(img)

## 3.3 Create Labelled Dataset

In [ ]:
# (anchor, positive) => 1,1,1,1,1
# (anchor, negative) => 0,0,0,0,0

In [ ]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [ ]:
samples = data.as_numpy_iterator()

In [ ]:
example = samples.next()

In [ ]:
example

## 3.4 Build Train and Test Partition

In [ ]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)

In [ ]:
res = preprocess_twin(*example)

In [ ]:
plt.imshow(res[1])

In [ ]:
res[2]

In [ ]:
#Build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [ ]:
#Training Partition
train_data = data.take(round(len(data)*0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(1)

In [ ]:
train_samples = train_data.as_numpy_iterator()

In [ ]:
train_samples= train_samples.next()

In [ ]:
len(train_samples[0])

In [ ]:
round(len(data)*0.3)

In [ ]:
# Testing Partition
test_data = data.skip(round(len(data)*0.7))
test_data = test_data.take(round(len(data)*0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

# 4. Model Engineering

## 4.1 Build Embedding Layer

In [ ]:
inp = Input(shape=(100, 100, 3), name='input_image')

In [ ]:
inp

In [ ]:
c1 = Conv2D(64, (10,10), activation='relu')(inp)

In [ ]:
c1

In [ ]:
m1 = MaxPooling2D(64,  (2, 2), padding='same')(c1)

In [ ]:
m1

In [ ]:
c2 = Conv2D(128, (7,7), activation='relu')(m1)
m2 = MaxPooling2D(128,  (2, 2), padding='same')(c2)

In [ ]:
c3 = Conv2D(128, (4,4), activation='relu')(m2)
m3 = MaxPooling2D(128,  (2, 2), padding='same')(c3)

In [ ]:
c4 = Conv2D(256, (4,4), activation='relu')(m3)
f1 = Flatten()(c4)
d1 = Dense(4096, activation='sigmoid')(f1)

In [ ]:
mod = Model(inputs=[inp], outputs=[d1], name='embedding')

In [ ]:
mod.summary()

In [ ]:
def make_embedding():
    inp = Input(shape=(100,100,3))
    
    #First Block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64,  (2, 2), padding='same')(c1)
    
    #Second BLock
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(128,  (2, 2), padding='same')(c2)
    
    #Third Block
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(128,  (2, 2), padding='same')(c3)
    
    #Final Embedding Block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [ ]:
embedding = make_embedding()

In [ ]:
embedding.summary()

## 4.2 Build Distance Layer

In [ ]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
        
    # Magic happens here
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
l1 = L1Dist()

In [ ]:
l1

## 4.3 Make Siamese Model

In [ ]:
input_image = Input(name = 'input_img', shape=(100, 100, 3))
validation_image = Input(name = 'validation_img', shape=(100, 100, 3))

In [ ]:
inp_embedding = embedding(input_image)
val_embedding = embedding(validation_image)

In [ ]:
siamese_layer = L1Dist()

In [ ]:
siamese_layer(inp_embedding, val_embedding)

In [ ]:
def make_siamese_model():
    
    #Handle Inputs
    input_image = Input(name = 'input_img', shape=(100, 100, 3))
    validation_image = Input(name = 'validation_img', shape=(100, 100, 3))
    
    #Combine Siamese Distance Component
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=[classifier], name='siameseNetwork')

In [ ]:
siamese_model = make_siamese_model()

In [ ]:
siamese_model.summary()

# 5. Training

## 5.1 Setup Loss and Optimizer

In [ ]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [ ]:
opt = tf.keras.optimizers.Adam(1e-4)

## 5.2 Establish Checkpoints

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

## 5.3 Build Train Step Function

In [ ]:
test_batch = train_data.as_numpy_iterator()

In [ ]:
batch_1 = test_batch.next()

In [ ]:
(batch_1[2])

In [ ]:
@tf.function
def train_step(batch):
    
    with tf.GradientTape() as tape:
        
        X = batch[:2]
        
        y = batch[2]
        
        #Forward Pass
        yhat = siamese_model(X, training=True)
        #Calculate loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
    #Calculate Gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    #Calculate updated weights
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    return loss

## 5.4 Build Training Loop

In [ ]:
def train(data, EPOCHS):
    #Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
    
        #Loop through each batch
        for idx, batch in enumerate(data):
        #Run train step here
            train_step(batch)
            progbar.update(idx+1)
            
        #Save checkpoint
        if epoch % 10 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

## 5.5 Train the model

In [ ]:
EPOCHS = 50

In [ ]:
train(train_data, EPOCHS)

# 6. Evaluate Model

## 6.1 Import Metrics

In [ ]:
from tensorflow.keras.metrics import Precision, Recall

## 6.2 Make Predictions

In [ ]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
test_var = test_data.as_numpy_iterator().next()

In [ ]:
len(test_var)

In [ ]:
len(test_var[0])

In [ ]:
# Make predictions
y_hat = siamese_model.predict([test_input, test_val])
y_hat

In [ ]:
#Post processing the results
[1 if prediction > 0.5 else 0 for prediction in y_hat]

In [ ]:
y_true 

In [ ]:
# Creatine a metric object
m = Recall()
# m = Precision()

# Calculate the recall value
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

## 6.4 Viz Results

In [ ]:
# Set plot size
plt.figure(figsize=(18,8))

# Set first subplot
plt.subplot(1,2,1)
plt.imshow(test_input[0]) 

# Set second subplot
plt.subplot(1,2,2)
plt.imshow(test_val[0])

# Renders cleanly
plt.show()

# 7. Save Model

In [ ]:
# Save weights
siamese_model.save('siamese_model.h5')

In [ ]:
# Relaod model
model = tf.keras.models.load_model('siamese_model.h5', 
                                   custom_objects={'L1Dist': L1Dist, 'BinaryCrossentropy': tf.losses.BinaryCrossentropy})

In [ ]:
# Make predictions with reloaded model
model.predict([test_input, test_val])

In [ ]:
# View model summary
model.summary()

# 8. Real TIme Test

## 8.1 Verification Function

In [ ]:
os.listdir((os.path.join('application_data', 'verification_images')))

In [ ]:
def verify(model, detection_threshold, verification_threshold):
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Make prediction
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
        
    # Detection Threshold
    detection = np.sum(np.array(results) > detection_threshold)
    
    #Verification threshold
    verification = detection / len(os.path.join('application_data', 'verification_images'))
    verified = verification > verification_threshold
    
    return results, verified 

## 8.2 OpenCV Real Time Verification

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250,:]
    
    cv2.imshow('Verification', frame)
    
    #Verification Trigger
    if cv2.waitKey(10) & 0XFF == ord('v'):
        # Save input image to application_data/input_image folder
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run Verification
        results, verified = verify(model, 0.65, 0.65)
        print(verified)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
np.sum(np.squeeze(results) > 0.5)

In [ ]:
model.summary()